In [13]:
import dash
from dash import html
from dash.dependencies import Input, Output
from markupsafe import Markup
from dash import dcc
import plotly.express as px
import pandas as pd
import numpy as np
from faker import Faker
import plotly.graph_objs as go
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMapWithTime
from folium.utilities import JsCode
from folium.features import GeoJsonPopup
from folium.plugins import TimeSliderChoropleth
from folium import plugins
from folium.plugins import HeatMap
import pickle
from folium import Map, Circle, Element
import pandas as pd
import json
from jinja2 import Template
from IPython.display import HTML, display

In [14]:
# Generate fake heatmap data
num_rows_heatmap = 1600
background_df = pd.DataFrame({
    'Latitude': np.random.uniform(50.5, 50.6, size=num_rows_heatmap),
    'Longitude': np.random.uniform(3, 3.2, size=num_rows_heatmap),
})
heat_data = [[[row['Latitude'], row['Longitude'], np.random.choice([0, 0.2, 1])] for _, row in background_df.iterrows()] for _ in range(24)]

In [15]:
# Sample data generation
np.random.seed(0)
fake = Faker()
num_rows = 1500
aed_placement_categories = ['outside', 'inside', 'difficult']
aed_df_mon = pd.DataFrame({
    'opening_hour': np.random.choice(range(7, 9), size=num_rows),
    'closing_hour': np.random.choice(range(17, 22), size=num_rows),
    'Latitude': np.random.uniform(50.7, 50.8, size=num_rows),
    'Longitude': np.random.uniform(4, 5, size=num_rows),
    'aed_placement': np.random.choice(aed_placement_categories, size=num_rows),
    'buffer_distance': np.random.choice([200, 150, 125], size=num_rows)
})

aed_df_tue = pd.DataFrame({
    'opening_hour': np.random.choice(range(7, 9), size=num_rows),
    'closing_hour': np.random.choice(range(17, 22), size=num_rows),
    'Latitude': np.random.uniform(50.5, 50.6, size=num_rows),
    'Longitude': np.random.uniform(3, 4, size=num_rows),
    'aed_placement': np.random.choice(aed_placement_categories, size=num_rows),
    'buffer_distance': np.random.choice([200, 150, 125], size=num_rows)
})



In [16]:
time_index = [f'{hour}:00' for hour in range(24)]

In [17]:
#base map creation
basemap = folium.Map(location=[50.5, 4.3517], zoom_start=8)

##layers of heatmaps
hm_mon = HeatMapWithTime(
        heat_data,
        index=time_index,
        auto_play=False,
        radius=80,
        min_opacity=0.3,
        gradient={0: 'red', 0.2: 'yellow', 1: 'green'}
    )

#Monday heat map layer
hm_mon.add_to(basemap)


basemap

In [18]:

m = Map(location=[50.5, 4.3517], zoom_start=8)
heat_data_0 = heat_data[0]

# layers of heatmaps
hm_mon_0 = HeatMap(
    heat_data_0,
    radius=80,
    min_opacity=0.3,
    gradient={0: 'red', 0.2: 'yellow', 1: 'green'}
)

# Monday heat map layer
hm_mon_0.add_to(m)

# Add circles from DataFrame
for index, row in aed_df_mon.iterrows():
    circle = Circle(
        location=[row['Latitude'], row['Longitude']],
        radius=row['buffer_distance'],  # Radius from DataFrame
        color='black',  # Initial color
        fill=True,
        fill_color='black',  # Initial fill color
        fill_opacity=0.7
    ).add_to(m)

# Serialize DataFrame to JSON
df_json = aed_df_mon.to_json(orient='records')
heat_data_json = json.dumps(heat_data)

template = Template("""
<!DOCTYPE html>
<html>
<head>
    <title>Map with Timeslider and AED Points</title>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <script src="https://unpkg.com/leaflet.heat/dist/leaflet-heat.js"></script>
</head>
<body>
    <div id="map" style="width: 100%; height: 100vh;"></div>
    <script>
        var df = {{ df_json | safe }};
        var heat_data = {{ heat_data_json | safe }};
        var heatmapLayer;
        var myMap;

        function createMap() {
            console.log("Creating map...");
            myMap = L.map('map').setView([50.5, 4.3517], 8);  // Initialize Leaflet map
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                attribution: 'Map data © <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'
            }).addTo(myMap);
            createSlider();
            updateHeatmap(0);  // Initialize heatmap at hour 0
            addCircles();  // Add circles to the map
        }

        function createSlider() {
            console.log("Creating slider...");
            var sliderContainer = document.createElement("div");
            sliderContainer.style.position = "fixed";
            sliderContainer.style.top = "10px";
            sliderContainer.style.left = "50%";
            sliderContainer.style.transform = "translateX(-50%)";
            sliderContainer.style.zIndex = "1000"; // Set a high z-index value;

            var slider = document.createElement("input");
            slider.type = "range";
            slider.min = 0;
            slider.max = 23; // 24 hours
            slider.value = 0;
            slider.step = 1;
            slider.id = "timeSlider";

            var label = document.createElement("label");
            label.id = "sliderLabel";
            label.innerHTML = "Time: 0:00";

            sliderContainer.appendChild(slider);
            sliderContainer.appendChild(label);
            document.body.appendChild(sliderContainer);

            slider.addEventListener("input", function() {
                var hour = parseInt(this.value);
                console.log("Slider value changed to:", hour);
                label.innerHTML = "Time: " + hour + ":00";
                updateHeatmap(hour);
                updateCircles(hour);
            });
        }

        function updateHeatmap(hour) {
            console.log("Updating heatmap...");
            var filteredData = heat_data[hour];
            if (heatmapLayer) {
                myMap.removeLayer(heatmapLayer);
            }
            heatmapLayer = L.heatLayer(filteredData, { 
                radius: 80, 
                blur: 25, 
                maxZoom: 17,
                gradient: {0: 'red', 0.2: 'yellow', 1: 'green'}  // Customize heatmap colors here
            }).addTo(myMap);
        }

        function addCircles() {
            console.log("Adding circles...");
            df.forEach(function(row) {
                var circle = L.circle([row['Latitude'], row['Longitude']], {
                    radius: row['buffer_distance'],  // Radius from DataFrame
                    color: 'black',  // Initial color
                    fill: true,
                    fillColor: 'black',  // Initial fill color
                    fillOpacity: 0.7
                }).addTo(myMap);
            });
        }

        function updateCircles(hour) {
            console.log("Updating circles...");
            var circles = document.querySelectorAll('.leaflet-interactive');
            circles.forEach(function(circle, index) {
                var color = getColorForHour(hour, index);
                circle.setAttribute('stroke', color);
                circle.setAttribute('fill', color);
            });
        }

        function getColorForHour(hour, index) {
            var openingHour = parseInt(df[index]['opening_hour']);
            var closingHour = parseInt(df[index]['closing_hour']);

            if (hour < openingHour || hour > closingHour) {
                return 'black'; // Before opening or after closing
            } else if (hour >= openingHour && hour <= closingHour) {
                return 'white'; // Between opening and closing
            } else {
                return 'purple'; // Shouldn't reach here, just in case
            }
        }

        document.addEventListener("DOMContentLoaded", function() {
            createMap();
        });
    </script>
</body>
</html>
""")

# Render the template with JSON data
html_code = template.render(df_json=df_json, heat_data_json=heat_data_json)

# Save the map to an HTML file
with open("map_with_timeslider_aed.html", "w") as f:
    f.write(html_code)